# Task 1. Password Uniqueness Check Using Bloom Filter

Create a function to check password uniqueness using a Bloom filter. This function should determine whether a password has been used before without the need to store the passwords themselves.

## Implementation

In [1]:
"""Password uniqueness check using Bloom filter implementation."""

import secrets
import string
from typing import Dict, List
import time

import mmh3

In [2]:
class BloomFilter:
    """A probabilistic data structure for fast membership queries.
    
    A Bloom filter is a space-efficient probabilistic data structure that is
    used to test whether an element is a member of a set. False positive matches
    are possible, but false negatives are not.
    """

    def __init__(self, size: int, num_hashes: int) -> None:
        """Initialize the Bloom filter.
        
        Args:
            size: The size of the bit array.
            num_hashes: The number of hash functions to use.
        """
        self.size = size
        self.num_hashes = num_hashes
        self.bit_array = [0] * size

    def _get_hash_indices(self, item: str) -> List[int]:
        """Generate hash indices for an item.
        
        Args:
            item: The item to hash.
            
        Returns:
            List of hash indices.
        """
        return [mmh3.hash(item, i) % self.size for i in range(self.num_hashes)]

    def add(self, item: str) -> None:
        """Add an item to the Bloom filter.
        
        Args:
            item: The item to add.
        """
        normalized_item = "" if item is None else str(item)
        for index in self._get_hash_indices(normalized_item):
            self.bit_array[index] = 1

    def contains(self, item: str) -> bool:
        """Check if an item might be in the set.
        
        Args:
            item: The item to check.
            
        Returns:
            True if the item might be in the set, False if it's definitely not.
        """
        return all(self.bit_array[index] == 1 
                  for index in self._get_hash_indices(item))

In [3]:
def generate_strong_password(length: int = 16) -> str:
    """Generate a cryptographically secure password.
    
    The password will contain at least one character from each category:
    lowercase, uppercase, digits, and punctuation.
    
    Args:
        length: The desired password length. Must be at least 4.
        
    Returns:
        A randomly generated password.
        
    Raises:
        ValueError: If length is less than 4.
    """
    if length < 4:
        raise ValueError("Password length must be at least 4")
    
    # Ensure at least one character from each required category
    required_chars = [
        secrets.choice(string.ascii_lowercase),
        secrets.choice(string.ascii_uppercase),
        secrets.choice(string.digits),
        secrets.choice(string.punctuation),
    ]
    
    # Fill remaining positions with random characters from all categories
    char_pool = string.ascii_letters + string.digits + string.punctuation
    remaining_chars = [
        secrets.choice(char_pool) for _ in range(length - len(required_chars))
    ]
    
    # Combine and shuffle all characters
    all_chars = required_chars + remaining_chars
    secrets.SystemRandom().shuffle(all_chars)
    
    return ''.join(all_chars)


def generate_passwords(count: int = 1, length: int = 16) -> List[str]:
    """Generate multiple unique passwords.
    
    Args:
        count: Number of passwords to generate.
        length: Length of each password.
        
    Returns:
        List of unique passwords.
    """
    passwords = []
    seen = set()
    
    # Generate unique passwords
    while len(passwords) < count:
        password = generate_strong_password(length)
        if password not in seen:
            seen.add(password)
            passwords.append(password)
    
    return passwords


def check_password_uniqueness(
    bloom_filter: BloomFilter, passwords: List[str]
) -> Dict[str, str]:
    """Check password uniqueness using a Bloom filter.
    
    Args:
        bloom_filter: The Bloom filter containing previously used passwords.
        passwords: List of passwords to check.
        
    Returns:
        Dictionary mapping passwords to their status ('Already used' or 'Available').
    """
    
    return {
        password: "Already used" if bloom_filter.contains(password) else "Available"
        for password in passwords
    }


def test_performance(pass_count: int = 1000, pass_length: int = 10) -> None:
    """Test performance with large datasets"""
    print("\nTest performance with large datasets:")

    # Create a large Bloom filter
    large_bloom = BloomFilter(size=10000, num_hashes=5)

    # Generate a large number of passwords
    start_time = time.time()
    large_password_set = generate_passwords(count=pass_count, length=pass_length)
    generation_time = time.time() - start_time

    # Add passwords to the filter
    start_time = time.time()
    for password in large_password_set:
        large_bloom.add(password)
    addition_time = time.time() - start_time

    # Check passwords
    test_passwords = large_password_set[: pass_count // 2] + generate_passwords(
        count=pass_count, length=pass_length
    )
    start_time = time.time()
    results = check_password_uniqueness(large_bloom, test_passwords)
    check_time = time.time() - start_time

    # Statistics of results
    already_used = sum(1 for status in results.values() if status == "Already used")
    available = len(results) - already_used

    print(
        f"Generation of {len(large_password_set)} passwords: {generation_time:.4f} sec"
    )
    print(
        f"Adding {len(large_password_set)} passwords to the filter: {addition_time:.4f} sec"
    )
    print(f"Checking {len(test_passwords)} passwords: {check_time:.4f} sec")
    print(f"Results of checking: {already_used} already used, {available} available")
    print(f"Memory usage of the filter: {len(large_bloom.bit_array)} bits")

## Results

In [4]:
def main() -> None:
    """Demonstrate password uniqueness checking with Bloom filter."""
    # Initialize Bloom filter
    bloom_filter = BloomFilter(size=1000, num_hashes=3)

    # Generate and add existing passwords to the Bloom filter
    PWD_CONT: int = 3
    PWD_NEW: int = 3
    PWD_LEN: int = 12
    existing_passwords = generate_passwords(count=PWD_CONT, length=PWD_LEN)
    existing_passwords += [
        None,
        "",
        " ",
        "  ",
        "Ω≤≥∑∏π",
        "123456",
        "!@#$%^&*()",
        "password123",
        "admin123",
        "qwerty123",
    ]

    print("Existing passwords:")
    for password in existing_passwords:
        print(f"  '{password}'")
        bloom_filter.add(password)

    # Generate new passwords to check (some existing + some new)
    new_passwords = generate_passwords(count=PWD_NEW, length=PWD_LEN)
    new_passwords += ["password123", "newpassword", "admin123", "guest"]
    passwords_to_check = existing_passwords[:PWD_CONT] + new_passwords  # Mix of old and new

    print(f"\nChecking {len(passwords_to_check)} passwords:")

    results = check_password_uniqueness(bloom_filter, passwords_to_check)

    # Display results
    for password, status in results.items():
        print(f"  '{password}' - {status}")

    # Run performance tests
    # test_performance(pass_count=1_000_000, pass_length=16)

if __name__ == "__main__":
    main()

Existing passwords:
  '\WFY2:q_48Si'
  'U6jj*{t&WzR*'
  'q4:&sr'XL?WX'
  'None'
  ''
  ' '
  '  '
  'Ω≤≥∑∏π'
  '123456'
  '!@#$%^&*()'
  'password123'
  'admin123'
  'qwerty123'

Checking 10 passwords:
  '\WFY2:q_48Si' - Already used
  'U6jj*{t&WzR*' - Already used
  'q4:&sr'XL?WX' - Already used
  ')o(%U!XC7y;v' - Available
  '602an\@Py_EP' - Available
  ')X.lx@M*q4pE' - Available
  'password123' - Already used
  'newpassword' - Available
  'admin123' - Already used
  'guest' - Available
